In [1]:
import pandas as pd
import requests , json , csv , time , re
from bs4 import BeautifulSoup as bs

In [5]:
Job_search_details = pd.DataFrame(columns={'Title', 'Url','Place', 'Time'})
Url = 'https://www.stepstone.de/5/job-search-simple.html?stf=freeText&ns=1&ke=Werkstudent&ws=essen&ra=30&li=100&wt=80002'
Request = requests.get(Url,'lxml')
HTML = bs(Request.content)
Links = HTML.find_all('article')
for i in range(len(Links)):
    job_title = Links[i].h2.text
    job_links = Links[i].find_all('a')
    job_url = 'https://www.stepstone.de'+job_links[1]['href']
    job_place_time = Links[i].find_all('li')
    job_place = job_place_time[1].text
    job_time = job_place_time[0].text
    Job_search_details = Job_search_details.append({
        'Title': job_title,
        'Url' : job_url,
        'Place' : job_place,
        'Time' : job_time
    }, ignore_index = True)

Full_Job_Des = pd.DataFrame()
for index , row in Job_search_details.iterrows():
    Job_req_des = dict()
    Title = row['Title']
    Place = row['Place']
    Time = row['Time']
    Links = row['Url']
    Request = requests.get(Links)
    Pages = bs(Request.content,'lxml')
    Job_Title = Pages.find('h1').text
    Main = Pages.find_all('main')
    if Main != None or Main != '':
        try:
            str_list = Main[1].text.split('\n')
        except:
            str_list = '[]'
            
        Texts = [x.replace('\xa0',' ') for x in str_list if x != '' and x !='\xa0' and x != ' ']
        
    print(Title)
    Full_Job_Des = Full_Job_Des.append({
                'Job Title': Job_Title,
                'Job Url' : Links,
                'Place' : Place,
                'Time' : Time,
                'Job Details': Texts
            },ignore_index = True)
Full_Job_Des.to_json('Full_Details.json')
Details = pd.read_json('Full_Details.json')

Werkstudent (m/w/d) Allrounder im Medienservice
Werkstudent (m/w/d) Health, Safety & Environment (HSE)
Werkstudent für Geodatenverarbeitung und -visualisierung in der Netzdokumentation (m/w/d)
Working student: Performance Management & Continous Improvement
Werkstudent Management Assistenz (m/w/d)
Werkstudent*in im Bereich Procurement Renewables
Working student Health, Safety and Environment & Sustainability (d/f/m)
Werkstudent Quality & Knowledge Management (m/w/d)
Werkstudent Project Management Office New Ways of Working m/w/d
Werkstudent Produktvertrieb & Beratung (w/m/d)
Working Student - Community Management & Marketing- #Interactive Communication
Werkstudent Research & Knowledge Management (m/w/d)
Working Student – IT Project Management m/f/t
Werkstudent Daily Accounting Finance (m/w/d)
Praktikant/Werkstudent Tax (w/m/d)
Praktikant/Werkstudent Legal (w/m/d)
Werkstudent (w/m/d) Grafik
Werkstudent HR ALDI Nord Akademie (m/w/d)
Praktikant / Werkstudent Management Consulting (w/m/d)
P

In [12]:
for i in range(len(Details)):
    print(Details['Place'][i])

Essen
Essen
Essen
Essen
Essen
Essen
Essen
Essen
Essen
Essen
Essen
Essen
Essen
Essen
Bielefeld, Bremen, Düsseldorf, Essen, Hamburg, Hannover, Kiel, Köln, Osnabrück, Saarbrücken
Bielefeld, Bremen, Düsseldorf, Essen, Hamburg, Hannover, Köln, Osnabrück, Saarbrücken
Essen
Essen
Berlin, Düsseldorf, Essen, Frankfurt a. M., Hamburg, Köln, München, Stuttgart
Berlin, Essen, Frankfurt a. M., Hamburg, Köln
Berlin, Duisburg, Düsseldorf, Essen, Frankfurt a. M., Hamburg, München
Düsseldorf, Essen, Hamburg, Stuttgart
Berlin, Duisburg, Düsseldorf, Essen, Frankfurt a. M., Hamburg, München
Berlin, Duisburg, Düsseldorf, Essen, Frankfurt a. M., Hamburg, München, Nürnberg, Stuttgart
Berlin, Düsseldorf, Essen, Frankfurt a. M., Hamburg, Köln, München, Nürnberg, Stuttgart
Essen
Essen
Essen
Essen
Düsseldorf
Essen
Essen
Essen
Dresden, Essen, Heilbronn, Freiburg, Köln oder Ravensburg
Essen
Essen
Düsseldorf, Essen, Köln, Osnabrück, Hamburg, Hannover
Duisburg, Düsseldorf, Essen, Hamburg, Hannover
Düsseldorf, Münche

In [11]:
Details['Place']

0                                                 Essen
1                                                 Essen
2                                                 Essen
3                                                 Essen
4                                                 Essen
                            ...                        
95                                           Düsseldorf
96    Düsseldorf, Essen, Duisburg, Mülheim an der Ru...
97                                           Düsseldorf
98                                           Düsseldorf
99                                           Düsseldorf
Name: Place, Length: 100, dtype: object